In [240]:
import os
import logging
import pandas as pd
import dtale
import numpy as np
import scipy.sparse as sp
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import chi2, VarianceThreshold, SelectKBest
from sklearn.cluster import KMeans, FeatureAgglomeration
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.compose import make_column_transformer, ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import eli5
from util import get_dialect
import homophone
import imp

logging.getLogger().setLevel(logging.INFO)
imp.reload(homophone)

<module 'homophone' from 'd:\\git\\sinetym\\scripts\\zhongguoyuyan\\homophone.py'>

In [230]:
class MyVarianceThreshold(VarianceThreshold):
    def get_feature_names(self, input_features=None):
        indices = self.get_support(indices=True)
        if input_features is None:
            return [f'x{i}' for i in indices]
        else:
            return [input_features[i] for i in indices]

class MyPipeline(Pipeline):
    def get_feature_names(self, input_features=None):
        feature_names = input_features
        for _, estimator in self.steps:
            feature_names = estimator.get_feature_names(feature_names)

        return feature_names

class MyColumnTransformer(ColumnTransformer):
    def get_feature_names(self, input_features=None):
        return [f'{name}_{n}' for name, transformer, col in self.transformers_ \
            for n in transformer.get_feature_names(input_features[col])]

In [100]:
prefix = r'D:\git\zhongguoyuyan\csv'
location = pd.read_csv(os.path.join(prefix, 'dialect', 'location.csv'), index_col=0)
char = pd.read_csv(os.path.join(prefix, 'words.csv'), index_col=0)
data = homophone.load_data(os.path.join(prefix, 'dialect'), location.index).reindex(char.index, axis=1, level=1)

2022-04-03 18:06:43,765 - INFO     - loading 1287 data files ...
2022-04-03 18:06:43,767 - INFO     - loading D:\git\zhongguoyuyan\csv\dialect\06K06mb01dz.csv ...
2022-04-03 18:06:43,803 - WARNING  - replace Ǿ -> ∅ 114
2022-04-03 18:06:43,819 - INFO     - loading D:\git\zhongguoyuyan\csv\dialect\06K10mb01dz.csv ...
2022-04-03 18:06:43,865 - WARNING  - replace Ǿ -> ∅ 115
2022-04-03 18:06:43,869 - WARNING  - replace ʨ -> tɕ 1
2022-04-03 18:06:43,890 - INFO     - loading D:\git\zhongguoyuyan\csv\dialect\06K09mb01dz.csv ...
2022-04-03 18:06:43,924 - WARNING  - replace Ǿ -> ∅ 116
2022-04-03 18:06:43,945 - INFO     - loading D:\git\zhongguoyuyan\csv\dialect\06K11mb01dz.csv ...
2022-04-03 18:06:44,004 - INFO     - loading D:\git\zhongguoyuyan\csv\dialect\06K07mb01dz.csv ...
2022-04-03 18:06:44,051 - WARNING  - replace Ǿ -> ∅ 141
2022-04-03 18:06:44,066 - INFO     - loading D:\git\zhongguoyuyan\csv\dialect\06K08mb01dz.csv ...
2022-04-03 18:06:44,097 - WARNING  - replace Ǿ -> ∅ 144
2022-04-03 1

In [103]:
train_loc, test_loc, train_data, test_data = train_test_split(location, data, test_size=0.2)
train_dialect = get_dialect(train_loc)
test_dialect = get_dialect(test_loc)

In [241]:
transformer = MyColumnTransformer(transformers=[(
    col,
    MyPipeline(steps=(
        ('phone_clustering', homophone.PhoneClustering(n_clusters=200)),
        ('homophone', homophone.Homophone()),
        ('variance_threshold', MyVarianceThreshold(threshold=0.998 * (1 - 0.998))),
        ('phomophone_clustering', homophone.HomophoneClustering(n_clusters=1000))
    )),
    train_data.columns.get_loc(col)
) for col in train_data.columns.levels[0]], n_jobs=3)
train_features = transformer.fit_transform(train_data)
feature_names = np.asarray(transformer.get_feature_names(char['item'].reindex(train_data.columns, level=1)))

In [231]:
phone_clustering = homophone.PhoneClustering(n_clusters=200)
hp = homophone.Homophone()
variance_threshold = MyVarianceThreshold(threshold=0.998 * (1 - 0.998))
homophone_clustering = homophone.HomophoneClustering(n_clusters=1000)

In [232]:
fea = phone_clustering.fit_transform(train_data['initial'])

In [233]:
fea = hp.fit_transform(fea)

In [236]:
fea = variance_threshold.fit_transform(fea)

In [238]:
fea = homophone_clustering.fit_transform(fea)

In [209]:
test_fea = phone_clustering.transform(test_data['initial'])

In [152]:
test_features = transformer.transform(test_data)

In [210]:
test_fea = hp.transform(test_fea)

In [212]:
test_fea = homophone_clustering.transform(test_fea)

In [245]:
idx = np.argsort(-np.asarray(train_features.var(axis=0)).squeeze())[:100]
df = pd.DataFrame(train_features[:, idx], index=train_data.index, columns=feature_names[idx])
df.insert(0, 'dialect', train_dialect)
dtale.show(df)

In [246]:
selector = SelectKBest(chi2, k=1000).fit(train_features[train_dialect != ''], train_dialect[train_dialect != ''])
idx = np.argsort(-selector.scores_)[:100]
df = pd.DataFrame(
    train_features[np.ix_(train_dialect != '', idx)],
    index=train_data[train_dialect != ''].index,
    columns=feature_names[idx]
)
df.insert(0, 'dialect', train_dialect[train_dialect != ''])
dtale.show(df, name='train_features')

In [247]:
lr = LogisticRegression(penalty='l1', solver='saga', fit_intercept=False).fit(train_features[train_dialect != ''], train_dialect[train_dialect != ''])
test_features = transformer.transform(test_data)
print(classification_report(train_dialect[train_dialect != ''], lr.predict(train_features[train_dialect != ''])))
print(classification_report(test_dialect[test_dialect != ''], lr.predict(test_features[test_dialect != ''])))

              precision    recall  f1-score   support

        东北官话       0.93      1.00      0.96        27
        中原官话       1.00      1.00      1.00        58
        兰银官话       1.00      1.00      1.00        12
        冀鲁官话       1.00      1.00      1.00        28
        北京官话       1.00      1.00      1.00        12
         吴方言       1.00      1.00      1.00        51
        客家方言       1.00      0.97      0.98        31
          平话       1.00      1.00      1.00         8
         徽方言       1.00      1.00      1.00         9
         晋方言       1.00      1.00      1.00        38
        江淮官话       1.00      1.00      1.00        36
        湖南土话       0.00      0.00      0.00         1
         湘方言       1.00      1.00      1.00        31
         粤方言       0.94      1.00      0.97        34
        胶辽官话       1.00      0.93      0.97        15
        西南官话       1.00      1.00      1.00       153
         赣方言       1.00      1.00      1.00        50
         闽方言       1.00    

In [248]:
eli5.explain_weights(lr, feature_names=feature_names)

Explanation(estimator="LogisticRegression(fit_intercept=False, penalty='l1', solver='saga')", description="\nFeatures with largest coefficients per class.\nCaveats:\n1. Be careful with features which are not\n   independent - weights don't show their importance.\n2. If scale of input features is different then scale of coefficients\n   will also be different, making direct comparison between coefficient values\n   incorrect.\n3. Depending on regularization, rare features sometimes may have high\n   coefficients; this doesn't mean they contribute much to the\n   classification result for most examples.\n", error=None, method='linear model', is_regression=False, targets=[TargetExplanation(target='东北官话', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='tone_区歪知资饥希州休优丢参蹲婚均章耕贞丰宫恭=捏_捏=拖车铺箍初胎开梯吹亏抄敲偷抽贪签添间天牵宽吞亲春汤糠更清轻青兄通充冲_捏=歌溪杯龟箫钩衫圈村仓疮筐窗灯僧坑钉厅葱翁蜂_捏=挖_捏=听_捏=师_捏=监_捏=墩_捏=多靴沙瓜花租乌猪书输该街低西鸡灰碑丝衣规追飞刀高包抓交焦烧腰修三甘尖心针深金音单肝安山班奸鲜先肩烟搬端酸官欢关弯砖翻冤根恩新身筋孙温分军熏帮张装霜伤姜秧光慌方桩双江冰升生争兵声星经东公冬松风中终封凶', weight=0.17910234663369354, std=None, value=None), FeatureWeight(feature='finals_下哑=鲜_夹甲鸭瞎=鲜_假嫁牙虾夏=鲜', weight=0.1530887182253987, std=None, value=None), FeatureWeight(feature='initial_渴=街解角', weight=0.14202436752153086, std=None, value=None), FeatureWeight(feature='finals_花化华画话划=网_网=瓜瓦挂抓', weight=0.1312612300905601, std=None, value=None), FeatureWeight(feature='finals_滑刮=网_挖=网_刷=网_袜=网', weight=0.12967590003967056, std=None, value=None), FeatureWeight(feature='tone_虱塞黑织哭屋=拥_搭鸽插鸭擦割八扎杀瞎歇刷刮托约郭剥桌拍拆=拥_拥=捏_拥=接贴切泼脱缺踢锡吃_拥=失息积惜击_拥=侧测_拥=七一出叔', weight=0.12374180794292673, std=None, value=None), FeatureWeight(feature='initial_僧=水', weight=0.11497811050645017, std=None, value=None), FeatureWeight(feature='finals_车蛇射=破婆磨磨摸_车蛇射=拨泼末_破婆磨磨摸=折撤舌设', weight=0.114711377279546, std=None, value=None), FeatureWeight(feature='tone_虱塞黑织哭屋=松_虱塞黑织哭屋=浆_虱塞黑织哭屋=杉_虱塞黑织哭屋=钢_虱塞黑织哭屋=烘_虱塞黑织哭屋=虾_虱塞黑织哭屋=拉_搭鸽插鸭擦割八扎杀瞎歇刷刮托约郭剥桌拍拆=松_搭鸽插鸭擦割八扎杀瞎歇刷刮托约郭剥桌拍拆=浆_搭鸽插鸭擦割八扎杀瞎歇刷刮托约郭剥桌拍拆=杉_搭鸽插鸭擦割八扎杀瞎歇刷刮托约郭剥桌拍拆=钢_搭鸽插鸭擦割八扎杀瞎歇刷刮托约郭剥桌拍拆=烘_搭鸽插鸭擦割八扎杀瞎歇刷刮托约郭剥桌拍拆=虾_搭鸽插鸭擦割八扎杀瞎歇刷刮托约郭剥桌拍拆=拉_接贴切泼脱缺踢锡吃=松_接贴切泼脱缺踢锡吃=浆_接贴切泼脱缺踢锡吃=杉_接贴切泼脱缺踢锡吃=钢_接贴切泼脱缺踢锡吃=烘_接贴切泼脱缺踢锡吃=虾_接贴切泼脱缺踢锡吃=拉_失息积惜击=松_失息积惜击=浆_失息积惜击=杉_失息积惜击=钢_失息积惜击=烘_失息积惜击=虾_失息积惜击=拉_松=七一出叔_浆=七一出叔_杉=七一出叔_钢=七一出叔_烘=七一出叔_虾=七一出叔_拉=七一出叔', weight=0.11388957640376168, std=None, value=None), FeatureWeight(feature='initial_项=宿_溪=宿_蟹=宿_鞋咸=宿', weight=0.11347129372128117, std=None, value=None), FeatureWeight(feature='finals_下哑=岩_夹甲鸭瞎=岩_岩=假嫁牙虾夏', weight=0.11265889918663624, std=None, value=None), FeatureWeight(feature='finals_拆窄摘=还_百拍白麦=还', weight=0.11146606303560457, std=None, value=None), FeatureWeight(feature='tone_松=侧测_浆=侧测_杉=侧测_闩=侧测_钢=侧测_侧测=烘_侧测=虾_侧测=拉', weight=0.10920393520035682, std=None, value=None), FeatureWeight(feature='tone_迟=夹_螺奴吴犁煤楼流林难兰连莲圆原园邻银轮娘黄王明营浓龙=夹_河图徐除锄如余怀池祠时随垂朝浮愁球严嫌寻琴弹延言完顽还传权贫民神辰勤唇纯坟尝房防狂朋能绳行程停形荣熊雄容=夹_渠=夹_蚊=夹_夹=华_夹=降_夹=茄婆爬茶斜蛇壶台财排牌柴鞋赔回皮骑棋锤肥桃桥条头绸潭蚕含咸钳甜沉钱缠田前盘还全船陈盆魂裙糖长床藤层棚平城瓶横蓬铜红虫穷_夹=锣鹅牙爷鱼来埋泥雷儿移眉梨姨围毛熬摇牛油南蓝岩盐颜棉年人门匀云忙迎名赢零脓', weight=0.09919110098094619, std=None, value=None), FeatureWeight(feature='tone_区歪知资饥希州休优丢参蹲婚均章耕贞丰宫恭=猫_猫=拖车铺箍初胎开梯吹亏抄敲偷抽贪签添间天牵宽吞亲春汤糠更清轻青兄通充冲_猫=歌溪杯龟箫钩衫圈村仓疮筐窗灯僧坑钉厅葱翁蜂_猫=摸_猫=听_猫=浆_猫=师_猫=闩_猫=监_猫=墩_猫=虾_猫=拉_猫=多靴沙瓜花租乌猪书输该街低西鸡灰碑丝衣规追飞刀高包抓交焦烧腰修三甘尖心针深金音单肝安山班奸鲜先肩烟搬端酸官欢关弯砖翻冤根恩新身筋孙温分军熏帮张装霜伤姜秧光慌方桩双江冰升生争兵声星经东公冬松风中终封凶', weight=0.09894935935759964, std=None, value=None), FeatureWeight(feature='initial_蚁=软', weight=0.09873026963220499, std=None, value=None)], neg=[FeatureWeight(feature='initial_如=软_入=软', weight=-0.14629959060973938, std=None, value=None), FeatureWeight(feature='tone_烛=憋_憋=雀_憋=窄_憋=客_憋=渴', weight=-0.1352798562275404, std=None, value=None), FeatureWeight(feature='tone_撤=摘_撤=客_撤=削_撤=拨', weight=-0.1200238493308059, std=None, value=None), FeatureWeight(feature='initial_松=宿', weight=-0.10051603788929435, std=None, value=None)], pos_remaining=122, neg_remaining=88), proba=None, score=None, weighted_spans=None, heatmap=None), TargetExplanation(target='中原官话', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='tone_区歪知资饥希州休优丢参蹲婚均章耕贞丰宫恭=日历木目_区歪知资饥希州休优丢参蹲婚均章耕贞丰宫恭=落麦_区歪知资饥

Executing shutdown due to inactivity...
Executing shutdown due to inactivity...


2022-04-05 09:48:32,609 - INFO     - Executing shutdown due to inactivity...


Executing shutdown...
Executing shutdown...


2022-04-05 09:48:36,741 - INFO     - Executing shutdown...
